In [1]:
# Import Dependencies

import pandas as pd
import csv
import sys, json
from sqlalchemy import create_engine
import psycopg2
import pymongo
import datetime
import numpy as np
np.random.seed(1)
import requests
from pprint import pprint
import matplotlib.pyplot as plt
from flask import jsonify
import company_domain
company_domain=company_domain.company_domain

# Extraction

### The path to CSV files to extract all Inc 5000 company list

In [2]:
# The path to our CSV files to extract all Inc 5000 company list

# 2019 Inc 5000 company list
Inc_2019_csv = "Resources/inc5000-2019.csv"
# 2018 Inc 5000 company list
Inc_2018_csv = "Resources/inc5000-2018.csv"
# 2007-2017 Inc 5000 company list
Inc_10year_csv ="Resources/inc5000_all10years.csv"


### Read the files

In [3]:
# Read csv file
# Read 2019 Inc 5000 company list
Inc_2019_df = pd.read_csv(Inc_2019_csv)

# Read 2018 Inc 5000 company list
Inc_2018_df = pd.read_csv(Inc_2018_csv)


# Read 2007-2017 Inc 5000 company list
Inc_10year_df = pd.read_csv(Inc_10year_csv,encoding='cp1252')

### Scraping Financial Times Americas Fastest Growing Companies 2020

In [4]:
# Financial times url to scrape
ft_url = 'https://www.ft.com/americas-fastest-growing-companies-2020'

In [5]:
# read the html file for scarping tables 
ft_tables= pd.read_html(ft_url)

In [6]:
# check it's type
type(ft_tables)

list

In [7]:
# Check how many tables are there 
len(ft_tables)

1

In [8]:
# scrape the the first table 
ft_tables_df = ft_tables[0]


In [70]:
# Transforming scraped data

# Drop column we don't need
drop_ft_columns=ft_tables_df.drop(['Revenue 2018 [in $m]','Revenue 2015 [in $m]','Number of employees 2018'], axis=1)


# Remove unnecessary sign "*" from the first column 

drop_ft_columns['Name'] = drop_ft_columns['Name'].str.replace(r'*', '')

# rename columns 
rename_ft_df=drop_ft_columns.rename(columns={"Rank":"rank","Name":"company_name","FT category":"industry","Absolute growth rate [in %]":"growth_rate","Founding Year":"founding_year","Compound annual growth rate (CAGR) [in %]":"compound_annual_growth_rate","Country":"country"})

# Add the year column 
rename_ft_df['rank_year']= '2020'

rename_ft_df['country']= 'United States'

# drop null values Growth

rename_ft_df=rename_ft_df.dropna(axis=1,how='all')

# remove duplicates and show clean data

clean_ft_df=rename_ft_df.drop_duplicates()
clean_ft_df.head()

,rank,company_name,country,industry,growth_rate,compound_annual_growth_rate,founding_year,rank_year
0,1,Niantic,United States,Games industry,180306.6,1117.4,2015,2020
1,2,UiPath,United States,Technology,37463.5,621.5,2005,2020
2,3,Publisher First (Freestar),United States,Advertising,36680.1,616.5,2015,2020
3,4,FreightWise,United States,Transport,30547.9,574.2,2015,2020
4,5,Veggie Noodle Co.,United States,Food & Beverage,24074.8,523.0,2015,2020


### API Call for Scraping Growjo top 1000 fastest growing companies in 2020.

In [72]:
# API call from Growjo 10,000 - The Fastest Growing Companies list
# We generated companies information based on domains. For example,
# you could retrieve a company’s name, location, employee, estimated revenue and job openings from their domain name. 


# The base url to request the Api call 
base_url = "https://growjo.com/api?url="

# The information we want to extract 
ranking = []
estimated_revenues = []
company_name = []
city = []
country = []
state = []
employees = []
founded = []
industry = []
total_funding = []

# a list of domains imported as company_domain you can find the list in company_domain.py in the main folder


# A for loop to get each domains and request information 
for domain in company_domain:
    target_url = base_url + domain
    data = requests.get(target_url)
    try:
        response = data.json()
    except ValueError:
        pass
# append all requested information letter to retrive
    ranking.append(response['ranking'])
    estimated_revenues.append(response['estimated_revenues'])
    company_name.append(response['company_name'])
    founded.append(response['founded'])
    city.append(response['city'])
    country.append(response['country'])
    state.append(response['state'])
    employees.append(response['employees'])
    industry.append(response['industry'])
    total_funding.append(response['total_funding'])
    

In [169]:
# Crerate and store the information in dictionary  
growjo_df = pd.DataFrame({
    "rank":ranking,
    "revenue_$":estimated_revenues,
    "company_name": company_name,
    "city":city,
    "founding_year":founded,
    "country":country,
    "state":state,
    "number_of_employees":employees,
    "industry":industry,
    "total_funding":total_funding
})

# Remove duplicate values
growjo_df=growjo_df.drop_duplicates()
#growjo_df.head()

In [170]:
# Transforming Growjo top 1000 fastest growing companies in 2020
# drop null values

growjo_df=growjo_df.dropna(axis=1,how='all')
# Add the year column 
growjo_df['rank_year']= '2020'


# Normalizing revenue in to mellions 
growjo_df['revenue_$']= (growjo_df['revenue_$']/1000000).apply(lambda x: '{:,.1f} Million'.format(x))


# remove duplicates and show clean data

clean_growjo_df=growjo_df.drop_duplicates()
clean_growjo_df.head()

,rank,revenue_$,company_name,city,founding_year,country,state,number_of_employees,industry,total_funding,rank_year
0,1.0,36.5 Million,LetsGetChecked,New York,2014.0,United States,NY,135,Hospital/Healthcare,$113M,2020
1,701.0,13.1 Million,100 Thieves,Los Angeles,2017.0,United States,CA,64,Entertainment,None,2020
2,832.0,97.3 Million,10X Genomics,Pleasanton,2012.0,United States,CA,628,Biotech,$242.6M,2020
3,124.0,29.9 Million,15Five,San Francisco,2011.0,United States,CA,206,Tech Services,$42.1M,2020
4,941.0,10.8 Million,2ULaundry,Charlotte,2015.0,United States,NC,43,Consumer,None,2020


# Transforming

### Cleaning 2019 Inc 5000 company list

In [171]:
# Drop column we don't need
drop_column_2019=Inc_2019_df.drop(['Profile','_ - previous_workers','_ - founded','_ - metro','url','_ - yrs_on_list'], axis=1)

# Add the year column 
drop_column_2019['rank_year']= '2019'

# Rename columns 
rename_df_2019= drop_column_2019.rename(columns={"_ - rank":"rank","name":"company_name","state":"state","_ - revenue":"revenue_$","_ - growth":"growth_rate","_ - industry":"industry","_ - workers":"number_of_employees","city":"city"})


# re-Order the data to suitable format

cleaned_2019_df =rename_df_2019[['rank','rank_year','company_name','industry','number_of_employees','revenue_$','growth_rate','city','state']]
#cleaned_2019_df.head()

### Cleaning 2018 Inc 5000 company list

In [172]:
# Drop column we don't need

drop_column_2018=Inc_2018_df.drop(['_ - id','_ - zipcode','_ - ifmid','_ - latitude','_ - longitude','_ - website','_ - state_l','_ - metrocode','_ - ifiid','_ - previous_workers','_ - metro','_ - founded','_ - url','_ - partner_lists - partner_lists','_ - yrs_on_list'], axis=1)

# Add the year column 
drop_column_2018['rank_year']= '2018'

# Rename columns 
rename_df_2018=drop_column_2018.rename(columns={"_ - rank":"rank","_ - company":"company_name","_ - state_s":"state","_ - revenue":"revenue_$","_ - growth":"growth_rate","_ - industry":"industry","_ - workers":"number_of_employees","_ - state_l":"state","_ - city":"city"})
rename_df_2018.head()

# Normalizing revenue in to mellions 
rename_df_2018['revenue_$']= (rename_df_2018['revenue_$']/1000000).apply(lambda x: '{:,.1f} Million'.format(x))


cleaned_2018_df =rename_df_2018[['rank','rank_year','company_name','industry','number_of_employees','revenue_$','growth_rate','city','state']]

#cleaned_2018_df.head()


### Cleaning 10 year (2007-2017) Inc 5000 company list

In [173]:
# Rename columns 

drop_column_10year_df=Inc_10year_df.drop(['_ - state_l','_ - metro','_ - yrs_on_list'],axis=1)

# Add the year column 
rename_10year_df=drop_column_10year_df.rename(columns={"year":"rank_year","_ - rank":"rank","_ - company":"company_name","_ - website":"company_Website","_ - state_s":"state","_ - revenue":"revenue_$","_ - growth":"growth_rate","_ - industry":"industry","_ - workers":"number_of_employees","_ - founded":"founded_Year","_ - city":"city"})


# Normalizing revenue in to mellions 
rename_10year_df['revenue_$']= (rename_10year_df['revenue_$']/1000000).apply(lambda x: '{:,.1f} Million'.format(x))

cleaned_10year_df =rename_10year_df[['rank','rank_year','company_name','industry','number_of_employees','revenue_$','growth_rate','city','state']]

# cleaned_10year_df.head()




### Combine all Inc 5000 data in to a single dataframe

In [174]:
# combine Inc 5000 data in to a single dataframe
combine_data =[cleaned_2018_df,cleaned_2019_df,cleaned_10year_df]
Inc_5000_df= pd.concat(combine_data)

In [175]:
# Add the year column 
Inc_5000_df['country']= 'United States'
Inc_5000_df.head()

,rank,rank_year,company_name,industry,number_of_employees,revenue_$,growth_rate,city,state,country
0,1.0,2018,SwanLeap,Logistics & Transportation,49.0,99.0 Million,75660.8425,Madison,WI,United States
1,2.0,2018,PopSockets,Consumer Products & Services,118.0,168.8 Million,71423.7620,Boulder,CO,United States
2,3.0,2018,Home Chef,Food & Beverage,865.0,255.0 Million,60165.5058,Chicago,IL,United States
3,4.0,2018,Velocity Global,Business Products & Services,55.0,49.2 Million,39816.5093,Denver,CO,United States
4,5.0,2018,DEPCOM Power,Energy,104.0,219.6 Million,38962.9022,Scottsdale,AZ,United States


In [88]:
# Merge all data in to a single dataframe
merge_all =[Inc_5000_df,clean_growjo_df,clean_ft_df]
merged_df= pd.concat(merge_all)
merged_df.head()

# Add unique id for company name 
id_df=merged_df.groupby(['company_name'], sort=True).ngroup().apply('{:006}'.format)

merged_df['id']=id_df

# Drop a duplicate value if there is any 
merged_df=merged_df.drop_duplicates('id')
merged_df=merged_df.dropna(subset=['company_name','id','revenue_$','rank','rank_year','company_name','industry','number_of_employees','city','state','country'])


merged_df.head()


,rank,rank_year,company_name,industry,number_of_employees,revenue_$,growth_rate,city,state,country,founding_year,total_funding,Rank_year,compound_annual_growth_rate,id
0,1.0,2018,SwanLeap,Logistics & Transportation,49.0,99.0 Million,75660.8425,Madison,WI,United States,NaN,NaN,NaN,NaN,020281
1,2.0,2018,PopSockets,Consumer Products & Services,118.0,168.8 Million,71423.7620,Boulder,CO,United States,NaN,NaN,NaN,NaN,016296
2,3.0,2018,Home Chef,Food & Beverage,865.0,255.0 Million,60165.5058,Chicago,IL,United States,NaN,NaN,NaN,NaN,009836
3,4.0,2018,Velocity Global,Business Products & Services,55.0,49.2 Million,39816.5093,Denver,CO,United States,NaN,NaN,NaN,NaN,022718
4,5.0,2018,DEPCOM Power,Energy,104.0,219.6 Million,38962.9022,Scottsdale,AZ,United States,NaN,NaN,NaN,NaN,005863


In [177]:
# Ready the tables in the way to load in postgres sql database

# company table 
Company = merged_df[['id','company_name','number_of_employees','industry','city','state','country']]
# drop null values from Id 
Company= Company.dropna(subset=['id'])
# drop duplicate id
Company = Company.drop_duplicates('id')
Company.set_index("id",inplace=True)

# Ranks table 
Ranks=merged_df[['id','rank','rank_year']]
Ranks.set_index("id",inplace=True)

# Groth table

Growth=merged_df[['id','growth_rate','total_funding','compound_annual_growth_rate','revenue_$']]
Growth.set_index("id",inplace=True)

# Load

In [178]:
# Create database connection
engine = create_engine('postgresql+psycopg2://postgres:216724401@2@localhost:5432/top-companies_db')
connection = engine.connect()


In [179]:
# confirm tables
engine.table_names()

['company', 'ranks', 'growth']

In [180]:
Company.to_sql(name='company',con=engine, if_exists='append',index=True)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "pk_company"
DETAIL:  Key (id)=(20281) already exists.

[SQL: INSERT INTO company (id, company_name, number_of_employees, industry, city, state, country) VALUES (%(id)s, %(company_name)s, %(number_of_employees)s, %(industry)s, %(city)s, %(state)s, %(country)s)]
[parameters: ({'id': '020281', 'company_name': 'SwanLeap', 'number_of_employees': 49.0, 'industry': 'Logistics & Transportation', 'city': 'Madison', 'state': 'WI', 'country': 'United States'}, {'id': '016296', 'company_name': 'PopSockets', 'number_of_employees': 118.0, 'industry': 'Consumer Products & Services', 'city': 'Boulder', 'state': 'CO', 'country': 'United States'}, {'id': '009836', 'company_name': 'Home Chef', 'number_of_employees': 865.0, 'industry': 'Food & Beverage', 'city': 'Chicago', 'state': 'IL', 'country': 'United States'}, {'id': '022718', 'company_name': 'Velocity Global', 'number_of_employees': 55.0, 'industry': '

In [ ]:
Ranks.to_sql(name='ranks',con=engine, if_exists='append',index=True)

In [ ]:
Growth.to_sql(name='growth',con=engine, if_exists='append',index=True)

In [ ]:
pd.read_sql_query('select * from company', con=engine).head()

In [ ]:
pd.read_sql_query('select * from ranks', con=engine).head()

In [ ]:
pd.read_sql_query('select * from growth', con=engine).head()